### This is a notebook to run the preprocessing pipeline prior to uploading your TIFs to deepcell

In [ ]:
# import required packages
import os
import numpy as np
import skimage.io as io
import xarray as xr
import sys

from segmentation.utils import data_utils

### <span style="color:#ff0000"> All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data' regardless of if it's input or output. </span>

In [ ]:
# load TIFs from GUI-based directory structure
data_dir = "../data/example_dataset/input_data/"
tiff_dir = os.path.join(data_dir, "single_channel_inputs/")
deepcell_dir = os.path.join(data_dir, "deepcell_input/")

# set this to true for multi-channel tiffs
MIBItiff = False

# data file suffix for low-level processed data
# only needed for MIBItiff = True
MIBItiff_suffix = "-MassCorrected-Filtered.tiff"

# nuclear channel name(s) (or nucs = None)
nucs = ['HH3']

# membrane channel name(s) (or mems = None)
mems = ['Membrane']

# validate data_dir
data_utils.validate_paths([data_dir,
                           tiff_dir,
                           deepcell_dir,
                           ])


### compute and filter point paths

In [ ]:
# either get all points in the folder
points = os.listdir(tiff_dir)
if MIBItiff:
    points = [point for point in points if point.endswith(MIBItiff_suffix)]
else:
    points = [point for point in points if os.path.isdir(tiff_dir + point)]

# optionally, select a specific set of points manually
# points = ["Point1", "Point2"]

# TODO: MIBItiff manual selection

### load images into notebook, process, and save as deepcell compatable input

In [ ]:
# load channels to be included in deepcell data
channels = (nucs if nucs else []) + (mems if mems else [])

# filter channels for None (just in case)
channels = [channel for channel in channels if channel is not None]

if MIBItiff:
    data_xr = data_utils.load_imgs_from_mibitiff(tiff_dir, mibitiff_files=points, channels=channels)
else:
    data_xr = data_utils.load_imgs_from_tree(tiff_dir, img_sub_folder="TIFs", fovs=points, imgs=channels)

# generate and save deepcell input tifs
data_utils.generate_deepcell_input(data_xr, deepcell_dir, nucs, mems)

## Deepcell upload instructions

Once your deepcell input images have been generated, zip all of your input images into a .zip file.

Navigate to [deepcell.org](https://deepcell.org) and select the 'PREDICT' option.

Under 'Job Type' select 'Multiplex'.  Then upload your .zip file, and press 'SUBMIT' to begin segmentation.